# Importing the Libraries

In [1]:
!pip install chart_studio

^C


In [2]:
# data operation libraries
import numpy as np
import pandas as pd

# importing visualisation libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for chloroplath plotting
import chart_studio.plotly as py
import plotly.graph_objs as go 
import plotly
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

# for datetime operations
import datetime as dt

# pandas general settings
pd.options.display.max_columns = None

ModuleNotFoundError: No module named 'chart_studio'

# Importing the Dataset

In [ ]:
data = pd.read_csv('../input/global-super-store-dataset/Global_Superstore2.csv', encoding='windows-1252')

# Data Preparation

In [ ]:
data.head(2) #taking a look at the dataframe structure

In [ ]:
# correcting 'Order Date' variable
data[['order_day','order_month','order_year']] = data['Order Date'].str.split('-', expand=True)
data['Order Date'] = data['order_year'] + '/' + data['order_month'] + '/' + data['order_day']
data['Order Date'] = pd.to_datetime(data['Order Date'])

In [ ]:
# doing likewise for 'Ship Date'
data[['ship_day','ship_month','ship_year']] = data['Ship Date'].str.split('-', expand=True)
data['Ship Date'] = data['ship_year'] + '/' + data['ship_month'] + '/' + data['ship_day']
data['Ship Date'] = pd.to_datetime(data['Ship Date'])

In [ ]:
# dropping the support columns
data.drop(columns=['order_day','order_month','order_year','ship_day','ship_month','ship_year'], inplace=True)

In [ ]:
data.info() #checkout the data types/ null rows and memory consumption

In [ ]:
# let's check out the columns which are suitable category column type

data.nunique()

In [ ]:
data['Ship Mode'] = data['Ship Mode'].astype('category')
data['Segment'] = data['Segment'].astype('category')
data['Country'] = data['Country'].astype('category')
data['Market'] = data['Market'].astype('category')
data['Region'] = data['Region'].astype('category')
data['Category'] = data['Category'].astype('category')
data['Sub-Category'] = data['Sub-Category'].astype('category')
data['Order Priority'] = data['Order Priority'].astype('category')

In [ ]:
data.info() #check the reduction in memory consumption

In [ ]:
# making sure neither of our category columns have leading spaces

def remove_leading_spaces(df):
    for cols in df.columns:
        if df[cols].dtypes in ['object','category']:
            df[cols] = df[cols].str.strip()
        return df

In [ ]:
data = remove_leading_spaces(data)

In [ ]:
data.head(2)

In [ ]:
# generating years from our 'Order_year' variable because we are going 
# to need this in future analysis

data['Order_year'] = data['Order Date'].dt.year

In [ ]:
# also total unique customer count is something we need in our future analysis

print('Number of unique customers made purchase in 2011: {}'.format(data[data['Order_year']==2011]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2012: {}'.format(data[data['Order_year']==2012]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2013: {}'.format(data[data['Order_year']==2013]['Customer Name'].nunique()))
print('Number of unique customers made purchase in 2014: {}'.format(data[data['Order_year']==2014]['Customer Name'].nunique()))

In [ ]:
def total_purchase_in_year(row):
    Order_year = row[24]
    
    if Order_year in [2011,2012,2013]:
        return 795
    else:
        return 794
    
    
# generating  'unique_customers_within_year' based on associated year value
# for that particular row

data['unique_customers_within_year'] = data.apply(total_purchase_in_year, axis='columns')

Before generating revenue column let's understand the intution behing Revenue.

>Revenue is another word for the amount of money a company generates from its sales.

>Revenue is most simply calculated as the number of units sold multiplied by the selling price.


<img src='https://stockanalysis.com/img/term/revenue-formula.png' width='300'/>

In [ ]:
# Generating 'Revenue' column
data['Revenue'] = data['Sales'] * data['Quantity']

# EDA

Solving the questions we have been asked

# Customers Analysis

**Question: 1**Profile the customers based on their frequency of purchase - calculate frequency of purchase for each customer

**Question: 2**Do the high frequent customers are contributing more revenue

**Question: 3**Are they also profitable - what is the profit margin across the buckets

**Question: 4**Which customer segment is most profitable in each year.

**Question: 5**How the customers are distributed across the countries- -

# Question:1

Profile the customers based on their frequency of purchase - calculate frequency of purchase for each customer

In [ ]:
purchase_frequency = data.groupby(['Order_year','Customer Name'])

In [ ]:
purchase_frequency.agg({'Customer Name': 'count',
                       'unique_customers_within_year': 'min',
                       'Revenue': 'sum',
                       'Profit': 'sum'}) 

In [ ]:
analysis_result = purchase_frequency.agg({'Customer Name': 'count',
                       'unique_customers_within_year': 'min',
                       'Revenue': 'sum',
                       'Profit': 'sum'})

In [ ]:
analysis_result.rename(mapper={'Customer Name': 'Purchase_during_year'}, axis=1, inplace=True)

Calculating Customer Purchase Frequency


The repeat purchase rate is a calculation that shows you the percentage of your current customer base that has purchased  at least a second time in a specific duration (usally take 365 days). This metric is influenced by your customer retention efforts and is a good indicator of the value you are providing your customers.

<img src='https://blog.smile.io/content/images/2020/05/How-to-Calculate-Purchase-Frequency---RPR-equation.png' width='300'/>

In [ ]:
analysis_result['Customer_purchase_frequency'] = analysis_result['Purchase_during_year']/analysis_result['unique_customers_within_year'] *100

# Answer:

Here we are only supposed to find the purchase frequency of each customer and not the one who are having the highest purchase frequency. So here is the result:

In [ ]:
analysis_result.head(5)

# Question:2

Do the high frequent customers are contributing more revenue?


The question here is comapring the high purchase frequency customers with high revenue generating customers. In the previous question we found out the purchase frequency of each customer, so out of those we will finf out highest purchse frequency customers for that year and then will compare to the highest revenue generator for that year.

In [ ]:
tmp_df = analysis_result.reset_index()

In [ ]:
tmp_df.head()

In [ ]:
grouped_object = tmp_df.groupby(['Order_year'])

In [ ]:
freq_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g,d in grouped_object:
    highest_freq_customers = d.nlargest(1, 'Customer_purchase_frequency')
    freq_df = pd.concat([freq_df, highest_freq_customers])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Customer_purchase_frequency']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Revenue','Profit']] = 'background-color: grey'
    return df 

In [ ]:
display(freq_df.style.apply(highlight_cols, axis = None))

In [ ]:
rev_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g,d in grouped_object:
    highest_rev_customers = d.nlargest(1, 'Revenue')
    rev_df = pd.concat([rev_df, highest_rev_customers])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Revenue']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Profit','Customer_purchase_frequency']] = 'background-color: grey'
    return df 

In [ ]:
display(rev_df.style.apply(highlight_cols, axis = None))

# Answer:


We can clearly see by comparing both the tables that neither of the high puchase frequency customers are there in the high revenue generating customer taable. 

So the answer is no, high purchase frequency customers aren't contributing to high revenue.

# Question 3

Are they also profitable - what is the profit margin across the buckets

In [ ]:
profit_df = pd.DataFrame(columns=tmp_df.columns)

In [ ]:
for g, d in grouped_object:
    highest_profit = d.nlargest(1, 'Profit')
    profit_df = pd.concat([profit_df,highest_profit])

In [ ]:
def highlight_cols(x): 
    df = x.copy()
    df.loc[:, ['Customer Name','Profit']] = 'background-color: green'
    df[['Order_year','Purchase_during_year','unique_customers_within_year','Revenue','Customer_purchase_frequency']] = 'background-color: grey'
    return df 

In [ ]:
display(profit_df.style.apply(highlight_cols, axis = None))

# Answer:

From the table above we can see that neither of the customers who were in high purchase frequency table or high revenue table are here in the high profitable customer table. May be these are the customer who are purchasing low quantity but the profit margin is higher on their purchase.

# Question 4:

Which customer segment is most profitable in each year.

In [ ]:
segment_group = data.groupby(['Order_year','Segment'])

In [ ]:
high_profit_df = segment_group.agg({'Profit':'sum'}).unstack()
high_profit_df.style.background_gradient(cmap='Spectral', subset=pd.IndexSlice[:, pd.IndexSlice[:,'Consumer']])

# Answer:

We can see that every year consumer segment is triggering more profit to the firm.

# Question: 4

How the customers are distributed across the countries?

In [ ]:
country_group = data.groupby(['Country'])

In [ ]:
customer_distribution = country_group.agg({'Customer ID':'count'})
customer_distribution.columns = ['Customer_count']
customer_distribution.reset_index(inplace=True)
customer_distribution

In [ ]:
country_map = dict(type='choropleth',
           locations=customer_distribution['Country'],
           locationmode='country names',
           z=customer_distribution['Customer_count'],
            reversescale = True,
           text=customer_distribution['Country'],
           colorscale='earth',
           colorbar={'title':'Customer Count'})

In [ ]:
layout = dict(title='Customer Distribution over Countries',
             geo=dict(showframe=False,projection={'type':'mercator'}))

# Answer: 

In [ ]:
choromap = go.Figure(data = [country_map],layout = layout)
iplot(choromap)

# Product Analysis

**Question: 1**Which country has top sales?

**Question: 2**Which are the top 5 profit-making product types on a yearly basis

**Question: 3**How is the product price varying with sales - Is there any increase in sales with the decrease in price at a day level

**Question: 4**What is the average delivery time across the counties - bar plot

# Question: 1

Which country has top sales?

In [ ]:
country_group = data.groupby('Country')

In [ ]:
country_sales = country_group.agg({'Sales':'sum'})
country_sales.sort_values(by='Sales', ascending=False)

# Answer: 


We can see that United States has top sales. Things are better when they are visually presented. Let's plot top 10 sales countries.

In [ ]:
import squarify

In [ ]:
top_10_sales = country_sales.nlargest(10, 'Sales')
top_10_sales.index

In [ ]:
plt.figure(figsize=(15,7))
revs = top_10_sales['Sales'].values
labels = ['United States: 2297200.8603',
         'Australia: 925235.853',
         'France: 858931.083',
         'China: 700562.025',
         'Germany: 628840.0305',
         'Mexico: 622590.61752',
         'India: 589650.105',
         'United Kingdom: 528576.3',
         'Indonesia: 404887.4979',
         'Brazil: 361106.41896']
squarify.plot(revs, label=labels,color= sns.color_palette('copper'), alpha=0.7)
plt.show()

# Question 2:

Which are the top 5 profit-making product types on a yearly basis

In [ ]:
year_category_group = data.groupby(['Order_year','Sub-Category'])

In [ ]:
year_category_proft_df = year_category_group.agg({'Profit':'sum'})
year_category_proft_df

In [ ]:
year_category_proft_df.reset_index(inplace=True)
category_yearly_profit = year_category_proft_df.groupby('Order_year')
top5_profit_category = pd.DataFrame(columns=year_category_proft_df.columns)

In [ ]:
for g, d in category_yearly_profit:
    high_profit_categories = d.nlargest(5, 'Profit')
    top5_profit_category = pd.concat([top5_profit_category,high_profit_categories])

# Answer: 

Below dataframe includes top 5 profit making products for each year.

In [ ]:
top5_profit_category.style.background_gradient(cmap='Spectral', subset=pd.IndexSlice[:, 'Profit'])

# Question 3:

How is the product price varying with sales - Is there any increase in sales with the decrease in price at a day level

**Note**: This question could have been more specifir to a coutry/product category and for a specific year but all we have asked is to see the trend between sales and price. Let's try to plot it as asked.

How to Calculate Unit Price. The unit price can be found using a simple formula if the quantity and total cost is known. Simply divide the total price by the quantity to find the unit price. Thus, the unit price is equal to the total price divided by the quantity.

In [ ]:
data['Unit_price'] = data['Sales']/data['Quantity']
data['Order_day'] = data['Order Date'].dt.day

# Answer:

From below chart we can see that when the prices are lower sales are high and when the prices increase, sales decrease.

In [ ]:
g5 = sns.FacetGrid(data, row = 'Order_day', col = 'Order_year', hue = 'Order_day')
kwe = dict(s = 50, linewidth = 0.5, edgecolor = 'black')
g5 = g5.map(plt.scatter, 'Unit_price', 'Sales')
g5.set(xlim=(0,100), ylim=(0,100))
for ax in g5.axes.flat:
    ax.plot((0,100),(0,100), c = 'gray', ls = '--')
g5.add_legend()

# Question: 4

What is the average delivery time across the counties - bar plot

In [ ]:
data['Delivery_duration'] = data['Ship Date']-data['Order Date']
country_group = data.groupby('Country')
delivery_duration_df = country_group.agg({'Delivery_duration':'mean'})
delivery_duration_df['Duration_in_hours'] = delivery_duration_df['Delivery_duration'] / dt.timedelta(hours=1)

We have 147 unique countries in our dataframe. Plotting them altogether on a same plot wouln't be possible. So I will only plot top 10 sales countries.

In [ ]:
delivery_duration_df

# Answer:

In [ ]:
top10_sales_country_DD =top_10_sales.merge(delivery_duration_df, how='left', left_index=True, right_index=True)
top10_sales_country_DD.reset_index(inplace=True)
top10_sales_country_DD.sort_values(by='Duration_in_hours')
# we can see that China and Brazil are providing fastest deliveries

In [ ]:
top10_sales_country_DD.iplot(kind='bar',x='Country', y='Duration_in_hours',
                            title= 'Countries And their Average Product Delivery Duration in Hours',
                            xTitle='Countries',
                            yTitle= 'AVG Delivery Duration in hours')